<a href="https://colab.research.google.com/github/kdenning/Political-ANN/blob/master/CIS_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing the data
import pandas as pd
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTl96SRjmGCo6hCFRYJzuP_v17kqx08ZC9fubES5aUOCVZjsHWXXxwPix2pvpF5WzOYyHb1SCbaEBDF/pub?output=csv'
pt_table = pd.read_csv(url, error_bad_lines=False, encoding='utf-8', delimiter=",")
pd.set_option('display.max_columns', None)
pt_table.head()

,subid,PoliOrient_1,VotePref,Day_Clin_In_PT,Day_Clin_In_C,Day_Clin_Out_PT,Day_Clin_Out_C,Day_Trump_Out_PT,Day_Trump_Out_C,Day_Trump_In_PT,Day_Trump_In_C
0,1,2,1,NaN,NaN,NaN,Jen gets up and feeds her 5 kids. She goes to ...,NaN,NaN,NaN,NaN
1,4,5,2,NaN,NaN,NaN,NaN,NaN,I BELIEVE A TYPICAL DAY FOR JEN IS FILLED WITH...,NaN,NaN
2,5,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"She wakes up early, gets ready for work, makes..."
3,6,1,1,NaN,Jen is a graudate student in Social work and t...,NaN,NaN,NaN,NaN,NaN,NaN
4,7,3,1,"A feminist goes about her day, ranting about h...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# getting data in long format
table_long = pd.melt(pt_table, id_vars=['subid', 'VotePref'], value_vars=['Day_Clin_In_PT', 'Day_Clin_In_C', 'Day_Clin_Out_PT', 'Day_Clin_Out_C', 'Day_Trump_Out_PT', 'Day_Trump_Out_C', 'Day_Trump_In_PT', "Day_Trump_In_C" ])
# removing NaN's
long_nonan = table_long.dropna()
# checking the data looks right and is the right number of rows; should be 773
print(long_nonan.head())
print(len(long_nonan))

    subid  VotePref        variable  \
4       7         1  Day_Clin_In_PT   
8      11         1  Day_Clin_In_PT   
18     21         1  Day_Clin_In_PT   
20     23         1  Day_Clin_In_PT   
27     30         1  Day_Clin_In_PT   

                                                value  
4   A feminist goes about her day, ranting about h...  
8   I am a gym teacher. My typical week begins wit...  
18  I think Jen is a very intelligent hard working...  
20  I wake up in the morning, take a shower then e...  
27                   She wants equality for everyone.  
773


In [10]:
# Getting puddles and spacey loaded
puddles = 'uo-puddles' 
clone_url = f'https://github.com/{puddles}/uo_puddles.git'
!git clone $clone_url 
import uo_puddles.uo_puddles as up

import spacy
!python -m spacy download en_core_web_md
import en_core_web_md
nlp = en_core_web_md.load()  

Cloning into 'uo_puddles'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 258 (delta 154), reused 64 (delta 33), pack-reused 0
Receiving objects: 100% (258/258), 66.97 KiB | 7.44 MiB/s, done.
Resolving deltas: 100% (154/154), done.
     |████████████████████████████████| 96.4MB 1.3MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=2dabbeb3ce295119690833afceee1e9fc44879f9ff89d21f4e02c585a909d1d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ji0gfnnh/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
# Shuffle table to prep for data split
import numpy as np
rsgen = np.random.RandomState(1234)

shuffled_table = long_nonan.sample(frac=1, random_state=rsgen).reset_index(drop=True)
shuffled_table.head()

# Pulling out strings
text = shuffled_table['value'].tolist()
vote = shuffled_table['VotePref'].tolist() # 1 = voted for Clinton, 2 = voted for Trump

In [0]:
# Getting Glove vectors for words in shuffled lists


In [0]:
# Split data into test and training sets